# G202158002 Shin Won Hyung

## Ex 5.1

$P(w | \theta) = \frac{1}{\theta} \exp(-\frac{1}{\theta}w)$

$\Rightarrow W = W(\theta) ~ \sim ~ Exp(\frac{1}{\theta})$

Let $W' ~ \sim ~ Exp(1)$, then $Q(\theta, V)$ is expressed in the equicalent form

$$Q(\theta, V) = f(\theta, Z) + \frac{1}{\theta} W'$$

and joint distribution for $\{Z, W'\}$ does not depend on $\theta$.

## Ex 5.4

Let $Q(\theta, V)$ is noisy observation of $L(\theta)$.

Then,

$$e(\theta) = Q(\theta, V) - L(\theta) = \theta^T W \theta + \theta^T w$$

By condition A.3 of statistics conditions for convergence,

$$E[e(\theta)] = 0$$

If $e(\theta) ~ \sim ~ N(0,1)$, then $E[e(\theta)] = 0$.

$$\theta^T W \theta + \theta^T w = \theta^T (W \theta + w)$$

Therefore, $W\theta + w ~ \sim ~ N(0, I_p)$.

## Ex 5.5

$$Q(\theta, V) = \sum_{i=1}^{p/2} t_i + \theta^T B \theta + \theta^T V$$

$$\frac{\partial Q(\theta, V)}{\partial \theta} = (B + B^T) \theta + \theta = (2B + I_p)\theta$$

## Ex 5.8

In [2]:
import numpy as np


def generate_x_pairs(max_sample_num=1000):
    c = np.random.randint(1, 11, size=max_sample_num)
    w = np.random.randint(11, 111, size=max_sample_num)
    return np.stack([w,c], 1)

def init_theta():
    return np.array([1., .5])


def generate_gain_seq(iter, start_gain):
    return start_gain / np.power(iter + 100, .501) 


def generate_noise_1():
    return np.random.normal(0, 5**2)


def compute_h(theta, x):
    lmbd, beta = theta
    w, c = x
    return lmbd * np.power(c, beta) * np.power(w, 1-beta)


def compute_euclidian(theta, optimal_theta):
    return np.sqrt(np.sum(np.power(theta - optimal_theta, 2)))


def compute_grad(theta, x):
    lmbd, beta = theta
    w, c = x
    grad_lmbd = w * np.power(c / w, beta)
    grad_beta = lmbd * w * np.power(c / w, beta) * np.log(c / w)
    
    return np.array([grad_lmbd, grad_beta])


def update_theta(theta, gain_seq, grad, x, z):
    h = compute_h(theta, x)
    return theta - gain_seq * grad * (h - z)


max_iter = 1000
max_repl = 5
optimal_theta = np.array([2.5, 0.7])

x = generate_x_pairs()

for seed in range(max_repl):
    theta = init_theta()
    np.random.seed(seed)
    np.random.shuffle(x)
    
    for iter, x_sample in enumerate(x):
        gain_seq = generate_gain_seq(iter+1, start_gain=0.00125)
        noise = generate_noise_1()

        z = compute_h(optimal_theta, x_sample) + noise
        grad = compute_grad(theta, x_sample)

        theta = update_theta(theta, gain_seq, grad, x_sample, z)
    print(f"[Case (i) / Replication {seed+1}] theta = {theta}")

print()

for seed in range(max_repl):
    theta = init_theta()
    np.random.seed(seed)
    np.random.shuffle(x)
    x_sum = np.array([0,0])
    z_sum = 0.
    for iter, x_sample in enumerate(x):
        gain_seq = generate_gain_seq((iter+1) // 2, start_gain=0.0075)
        noise = generate_noise_1()

        z = compute_h(optimal_theta, x_sample) + noise
        if (iter + 1) % 2 != 0:
            x_sum += (x_sample)
            z_sum += z
            continue
        
        x_mean = x_sum / 2
        z_mean = z_sum / 2
        grad = compute_grad(theta, x_mean)

        theta = update_theta(theta, gain_seq, grad, x_mean, z_mean)

        x_sum = np.array([0,0])
        z_sum = 0.
    print(f"[Case (ii) / Replication {seed+1}] theta = {theta}")

print()

for seed in range(max_repl):
    theta = init_theta()
    np.random.seed(seed)
    np.random.shuffle(x)
    for epoch in range(10):
        for iter, x_sample in enumerate(x):
            gain_seq = generate_gain_seq(epoch*1000 + iter+1, start_gain=0.00015)
            noise = generate_noise_1()

            z = compute_h(optimal_theta, x_sample) + noise
            grad = compute_grad(theta, x_sample)

            theta = update_theta(theta, gain_seq, grad, x_sample, z)
    print(f"[Case (iii) / Replication {seed+1}] theta = {theta}")

[Case (i) / Replication 1] theta = [1.99178932 0.67422034]
[Case (i) / Replication 2] theta = [0.42632878 4.34593277]
[Case (i) / Replication 3] theta = [-1.04199602  9.85206361]
[Case (i) / Replication 4] theta = [0.30418348 7.13508892]
[Case (i) / Replication 5] theta = [0.10929207 3.89262157]

[Case (ii) / Replication 1] theta = [-5.44028019 28.40830444]
[Case (ii) / Replication 2] theta = [1.18017766 1.8399834 ]
[Case (ii) / Replication 3] theta = [1.97344876 4.56361152]
[Case (ii) / Replication 4] theta = [-3.53318884 22.14324123]
[Case (ii) / Replication 5] theta = [0.33448318 3.81143552]

[Case (iii) / Replication 1] theta = [1.64024194 0.55366367]
[Case (iii) / Replication 2] theta = [1.65650074 0.52887176]
[Case (iii) / Replication 3] theta = [1.63403321 0.52254819]
[Case (iii) / Replication 4] theta = [1.59656676 0.51038819]
[Case (iii) / Replication 5] theta = [1.65923174 0.52017671]
